# Degrees

### Write a program that determines how many “degrees of separation” apart two actors are.

In [18]:
import sys
import csv

from util import Node, StackFrontier, QueueFrontier

In [10]:
# Maps names to a set of corresponding person_IDs
names = {}

# Maps person_IDs toa  dictionary of: name, birth, movies (a set of movie_IDs)
people = {}

# Maps movie_IDs to a dictionary of: title, year, starts (a set of person_IDs)
movies = {}

In [11]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

In [12]:
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

In [13]:
class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [ ]:
### Write a program that determines how many “degrees of separation” apart two actors are.

def load_data(directory):
    """
    Load data from CSV files into memory from the given directory.
    """
    
    # Load people
    with open(f"{directory}/people.csv", encoding = "utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name" : row["name"],
                "birth" : row["birth"],
                "movies" : set(),
            }
            
            # If the name is not in the list, add it to the list
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            # If the name is in the list, add the ID to the set
            else:
                names[row["name"].lower()].add(row["id"])
    
    # Load movies
    with open(f"{directory}/movies.csv", encoding = "utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title" : row["title"],
                "year" : row["year"],
                "stars" : set(),
            }
            
    # Load stars
    with open(f"{directory}/stars.csv", encoding = "utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


In [ ]:
def main():
    if len(sys.argv) > 2:
        sys.exit("Usage: python degrees.py [directory]")
    directory = sys.argv[1] if len(sys.argv) == 2 else "large"
    
    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")
    
    # Prompt user for name
    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")
        
    path = shortest_path(source, target)
    
    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")


In [ ]:
def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs 
    that connect the source to the target.
    
    If no path possible, returns None.
    """
    # Initialize frontier to just the starting position
    start = Node(state = source, parent = None, action = None)
    frontier = QueueFrontier()
    frontier.add(start)
    
    # Initialize an empty explored set
    explored = set()
    
    # Keep looping until solution found
    while True:
        # If nothing left in frontier, then no path
        if frontier.empty():
            return None
        
        # Choose a node from the frontier
        node = frontier.remove()
        
        # If node is the goal, then we have a solution
        if node.state == target:
            actions = []
            cells = []
            
            while node.parent is not None:
                # Add the action and state to the lists
                actions.append(node.action)
                cells.append(node.state)
                # Move to the next node
                node = node.parent
            
            # Reverse the lists to get the correct order
            actions.reverse()
            cells.reverse()
            solution = []
            
            # Add the actions and cells to the solution
            # and return the solution
            for i in range(len(actions)):
                solution.append((actions[i], cells[i]))
            
            return solution
        
        # Mark node as explored
        explored.add(node.state)
        
        # Add neighbors to frontier
        for action, state in neighbors_for_person(node.state):
            if not frontier.contains_state(state) and state not in explored:
                child = Node(state = state, parent = node, action = action)
                frontier.add(child)


In [ ]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]